In [1]:
1+1

2

In [3]:
import os
import getpass
from uuid import uuid4
from typing import List

# LangChain 관련 임포트
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_core.documents import Document

# FAISS 임포트
import faiss

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
from dotenv import load_dotenv

load_dotenv()

# 환경 변수 설정
def setup_environment():
    """OpenAI API 키 설정"""
    if not os.environ.get("OPENAI_API_KEY"):
        os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key를 입력하세요: ")


def load_and_split_pdf(file_path: str) -> List[Document]:
    """
    PDF 파일을 로드하고 텍스트를 청크로 분할

    Args:
        file_path (str): PDF 파일 경로

    Returns:
        List[Document]: 분할된 문서 청크 리스트
    """
    # PDF 로더 초기화
    loader = PyPDFLoader(file_path)

    # PDF 문서 로드
    documents = loader.load()
    print(f"로드된 페이지 수: {len(documents)}")

    # 텍스트 분할기 초기화
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # 각 청크의 최대 크기
        chunk_overlap=200,  # 청크 간 겹치는 부분
        length_function=len,  # 길이 계산 함수
        separators=["\n\n", "\n", " ", ""],  # 분할 기준
    )

    # 문서를 청크로 분할
    split_documents = text_splitter.split_documents(documents)
    print(f"분할된 청크 수: {len(split_documents)}")

    return split_documents


def create_vector_store(documents: List[Document]) -> FAISS:
    """
    FAISS 벡터 스토어 생성 및 문서 임베딩

    Args:
        documents (List[Document]): 임베딩할 문서 리스트

    Returns:
        FAISS: 생성된 벡터 스토어
    """
    # OpenAI 임베딩 모델 초기화
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large"  # 최신 임베딩 모델 사용
    )

    # 임베딩 차원 크기 계산
    dimension_size = len(embeddings.embed_query("hello world"))
    print(f"임베딩 차원 크기: {dimension_size}")

    # FAISS 인덱스 생성 (L2 거리 기반)
    index = faiss.IndexFlatL2(dimension_size)

    # FAISS 벡터 스토어 초기화
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )

    # 문서를 벡터 스토어에 추가
    # 각 문서에 고유 ID 생성
    document_ids = [str(uuid4()) for _ in range(len(documents))]
    vector_store.add_documents(documents=documents, ids=document_ids)

    print(f"벡터 스토어에 {len(documents)}개 문서 추가 완료")

    return vector_store


def create_rag_chain(vector_store: FAISS) -> RetrievalQA:
    """
    RAG (Retrieval-Augmented Generation) 체인 생성

    Args:
        vector_store (FAISS): 벡터 스토어

    Returns:
        RetrievalQA: RAG 체인
    """
    # ChatOpenAI 모델 초기화
    llm = ChatOpenAI(
        model="gpt-4o-mini",  # 또는 "gpt-4"
        temperature=0.1,  # 창의성 조절 (0에 가까울수록 일관된 답변)
        max_tokens=1000,  # 최대 토큰 수
    )

    # 벡터 스토어를 리트리버로 변환
    retriever = vector_store.as_retriever(
        search_type="similarity",  # 유사도 검색
        search_kwargs={"k": 3},  # 상위 3개 문서 검색
    )

    # RetrievalQA 체인 생성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # 검색된 문서를 모두 컨텍스트로 사용
        retriever=retriever,
        return_source_documents=True,  # 소스 문서도 함께 반환
    )

    return qa_chain


def query_documents(qa_chain: RetrievalQA, question: str):
    """
    문서에 대한 질문 수행

    Args:
        qa_chain (RetrievalQA): RAG 체인
        question (str): 질문
    """
    print(f"\n질문: {question}")
    print("-" * 50)

    # 질문 실행
    result = qa_chain({"query": question})

    # 답변 출력
    print(f"답변: {result['result']}")

    # 참조된 소스 문서 출력
    if result.get("source_documents"):
        print(f"\n참조된 문서 수: {len(result['source_documents'])}")
        for i, doc in enumerate(result["source_documents"]):
            print(f"\n[문서 {i+1}]")
            print(f"페이지: {doc.metadata.get('page', 'N/A')}")
            print(f"내용 미리보기: {doc.page_content[:200]}...")


def save_vector_store(vector_store: FAISS, save_path: str = "faiss_index"):
    """
    벡터 스토어를 로컬에 저장

    Args:
        vector_store (FAISS): 저장할 벡터 스토어
        save_path (str): 저장 경로
    """
    vector_store.save_local(save_path)
    print(f"벡터 스토어가 '{save_path}'에 저장되었습니다.")


def load_vector_store(load_path: str = "faiss_index") -> FAISS:
    """
    저장된 벡터 스토어 로드

    Args:
        load_path (str): 로드할 경로

    Returns:
        FAISS: 로드된 벡터 스토어
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vector_store = FAISS.load_local(
        load_path, embeddings, allow_dangerous_deserialization=True
    )
    print(f"벡터 스토어가 '{load_path}'에서 로드되었습니다.")
    return vector_store


def main():
    """메인 실행 함수"""
    # 1. 환경 설정
    # setup_environment()

    # 2. PDF 파일 로드 및 분할
    pdf_file_path = "sample_02_ocr.pdf"  # PDF 파일 경로

    try:
        documents = load_and_split_pdf(pdf_file_path)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {pdf_file_path}")
        return
    except Exception as e:
        print(f"PDF 로드 중 오류 발생: {e}")
        return

    # 3. 벡터 스토어 생성
    print("\n벡터 스토어 생성 중...")
    vector_store = create_vector_store(documents)

    # 4. 벡터 스토어 저장 (선택사항)
    save_vector_store(vector_store)

    # 5. RAG 체인 생성
    print("\nRAG 체인 생성 중...")
    qa_chain = create_rag_chain(vector_store)

    # 6. 질문 예시
    sample_questions = [
        "이 문서의 주요 내용은 무엇인가요?",
        "문서에서 가장 중요한 포인트는 무엇인가요?",
        "특정 키워드나 개념에 대해 설명해주세요.",
    ]

    # 7. 질문 실행
    for question in sample_questions:
        query_documents(qa_chain, question)
        print("\n" + "=" * 80 + "\n")

    # 8. 대화형 질문 모드
    print("대화형 질문 모드입니다. 'quit'을 입력하면 종료됩니다.")
    while True:
        user_question = input("\n질문을 입력하세요: ")
        if user_question.lower() in ["quit", "exit", "종료"]:
            break

        if user_question.strip():
            query_documents(qa_chain, user_question)


main()

로드된 페이지 수: 5
분할된 청크 수: 5

벡터 스토어 생성 중...
임베딩 차원 크기: 3072
벡터 스토어에 5개 문서 추가 완료
벡터 스토어가 'faiss_index'에 저장되었습니다.

RAG 체인 생성 중...

질문: 이 문서의 주요 내용은 무엇인가요?
--------------------------------------------------
답변: 이 문서의 주요 내용은 "청년일자리 도약장려금" 정책에 대한 정보입니다. 이 정책은 기업의 청년 고용 확대를 지원하고 취업 애로 청년의 취업을 촉진하기 위해 마련되었습니다. 주요 내용으로는 청년을 정규직으로 채용하고 6개월 이상 고용 유지 시 기업에 지원금을 지급하는 내용이 포함되어 있습니다. 지원금은 유형에 따라 다르며, 5인 이상 우선지원대상기업에서 취업 애로 청년을 채용할 경우 최대 720만원을 지원하고, 18개월 이상 재직한 청년에게는 장기근속 인센티브도 지급됩니다. 사업 운영 기간은 2025년 1월 1일부터 12월 31일까지이며, 신청 기간도 동일합니다.

참조된 문서 수: 3

[문서 1]
페이지: 4
내용 미리보기: ■ 기타
기타 유익 정보
주관 기관 고용노동부
운영 기관
참고 사이트1 https://www.work24.go.kr
참고 사이트2
온통청 년 (www.youthcenter.go.kr)
5페이지
출력일자 : 2025-07-14...

[문서 2]
페이지: 3
내용 미리보기: ■ 신청 방법
신청 절차
신청방법 및 지원절차＜br＞（참여신청） 기업이 운영기관에 채용계획 제출（전 
산시스템 활용）-〉운영기관의 검토 및 승인-〉운영기관-기업 간 지원협약 체 
결＜br＞（채용） 기업은 청년 채용 후 운영기관에 채용명단 제출，6개월 간 고 
용 유지하며 임금지급＜br＞（장려금 신청） 최소 근속기간 도래 후 운영기관에 
장려금 신청 ＜br〉...

[문서 3]
페이지: 0
내용 미리보기: 청년정책 정보
■ 정책 기본정보
정책명 청년일자리 도약장려금
정책소개
기업의 청년고용 확대를 지원하고，취